# Worker

Bridgic is an innovative framework for agent programming and dynamic task orchestration. It offers developers a brand-new programming paradigm. Whether it's for rapid experimentation, complex system orchestration, or building intelligent agents with long lifecycles, Bridgic provides concise and powerful support.

In this tutorial, let's build a sample word learning assistant. Input a word, output its derivational variations and make sentences using these variations. At the same time, we will understand the sample usage of a bridgic project along with the word learning assistant.

## Getting Started

Before we start, let's prepare the running environment.

In [1]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

_api_key = os.environ.get("OPENAI_API_KEY")
_model_name = os.environ.get("OPENAI_MODEL_NAME")  # Using gpt-4.1-mini in this example

In [2]:
# Import the necessary packages.
from bridgic.llms.openai.openai_llm import OpenAILlm, Message, Role
from bridgic.core.automa import GraphAutoma, worker

### 1. Initialize a model.

Bridgic provides a powerful encapsulation for model usage. Here, we will simply use it first. If you want to know more details, you can refer to this tutorial: [Llm](llm.ipynb).

In [3]:
llm = OpenAILlm(
    api_key=_api_key,
    timeout=20,
)

### 2. Complete chatbot

There are two steps to complete the word learning assistant:

1. Generate derivatives of the input word.
2. Make sentences with derivatives.

In [9]:
class WordLearningAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_derivatives(self, word: str):
        print(f"------Generating derivatives for {word}------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Generate derivatives of the input word in a list.", role=Role.SYSTEM),
                Message.from_text(text=word, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of generating derivatives------\n")
        return response.message.content

    @worker(dependencies=["generate_derivatives"], is_output=True)
    async def make_sentences(self, derivatives):
        print(f"------Making sentences with------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Make sentences with the input derivatives in a list.", role=Role.SYSTEM),
                Message.from_text(text=derivatives, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of making sentences------\n")
        return response.message.content

Let's run it!

In [10]:
word_learning_assistant = WordLearningAssistant()
res = await word_learning_assistant.arun(word="happy")

------Generating derivatives for happy------
1. happiness  
2. happily  
3. happier  
4. happiest  
5. unhappiness  
6. unhappily  
7. unhappier  
8. unhappiest
------End of generating derivatives------

------Making sentences with------
1. Happiness is the key to a fulfilling life.  
2. She happily accepted the invitation to the party.  
3. I'm happier now that I've started a new job.  
4. That was the happiest day of my childhood.  
5. His unhappiness became apparent after the breakup.  
6. They lived unhappily together for many years.  
7. After the storm, the plants looked unhappier than before.  
8. She felt the unhappiest when she lost her favorite book.
------End of making sentences------



> Note: Ensure you have set up your .env file to store your OPENAI_API_KEY or set up your terminal environment variable. This key is necessary for authenticating requests to the OpenAI API.

Get it! We successfully completed the word learning assistant. It correctly completed the task as per our requirements.

## Understand

The above example is the typical usage of writing an agent application with Bridgic. Now let's understand some of its components.

### Worker

Any callable object (such as functions, methods, etc.) when used by the framework, will be converted into a worker object and serve as the **smallest execution unit** for scheduling and orchestration. 

> Note: From the perspective of the Bridgic framework, a worker must be placed in an automa for scheduling before it can be executed. Of course, even after packaging it as a worker, you can directly call `worker.arun()` or `worker.run()` to run it, but this is not within the purview of bridgic.

There are two ways to convert a callable object into a Worker object. There are two ways to convert a callable object into a worker object. The first is to use the `@worker` decorator, and the second is to add it through the `add_worker()` interface.

#### 1. @worker

Just as in the example of the word learning assistant, we can use decorator syntax to wrap functions and methods into a worker object.

In [ ]:
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def start(self, x: int):
        return x    

Or, you can also use it like this：

In [ ]:
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

@my_automa.worker(is_start=True)
async def start(automa, x: int):
    return x

Both of these two ways can correctly add the `start` to `MyAutoma`.





### Automa

Automa is a specific worker. Automa serves as the **orchestration engine**. Developers can entrust multiple Workers to Automa for management, and it is responsible for unified scheduling and operation, acting as the entry point for the entire process.